In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2


In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from query.query_functions2 import count_from_triples


def get_query_counts(pred_querries):
    querries_with_count = list()

    for k, v in pred_querries.items():
        if k == "invalids":
            continue
        elif k == "backward2":
            querries_with_count.extend(v)
        else:
            if len(v) == 0:
                continue
            for simple_query in v:
                querries_with_count.append([[simple_query], count_from_triples(simple_query)])
    return querries_with_count


def query_overlap(all_triples, smaller):
    smaller_bools = list()

    for triple in smaller:
        less_go = False
        for rem in all_triples:
            cool = True
            for i in range(len(triple)):
                rem = [elem.replace("<", "").replace(">", "") for elem in rem]
                if triple[i][0] == "?" and rem[i][0] == "?":
                    continue
                if triple[i] != rem[i]:
                    cool = False
                    break
            if cool:
                less_go = True
                break
        smaller_bools.append(less_go)

    return all(smaller_bools)

In [3]:
import pickle
from evaluation.evaluate_script import graph_eval_ranking_first
import os
import json

file = "/home/kardosp/continuethelist/outputs/graphwalk_v4_output_sample4_v1.pickle"

with open(file, "rb") as f:
    algo_results = pickle.load(f)

In [4]:
x = algo_results[0]

x.keys()

dict_keys(['_id', 'corrected_question', 'intermediary_question', 'sparql_query', 'sparql_template_id', 'result', 'graph', 'result_cleaned', 'result_urlonly', 'seed', 'gold', 'output', 'explanation'])

In [34]:
from tqdm import tqdm
from evaluation.evaluation_script_v2 import graph_ranking_first_nodes

pred_column = "explanation"
ranks = list()
extra_info = True

query_sizes = list()
for record in tqdm(algo_results):
    # Filter out Nones
    if isinstance(record[pred_column], list):
        continue
    if extra_info:
        record_rank, _, query_size_single = graph_ranking_first_nodes(record["graph"],
                                                                   record[pred_column],
                                                                   extra_info)
        query_sizes.append(query_size_single)
    else:
        record_rank = graph_ranking_first_nodes(record["graph"], record[pred_column])
    ranks.append(record_rank)


100%|█████████████████████████████████████████| 250/250 [00:20<00:00, 12.44it/s]


In [35]:
ranks

[2,
 2,
 3,
 1,
 3,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 2,
 1,
 -1,
 -1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 -1,
 6,
 2,
 11,
 -1,
 1,
 1,
 -1,
 2,
 1,
 1,
 2,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 -1,
 2,
 1,
 4,
 1,
 1,
 1,
 10,
 -1,
 52,
 2,
 3,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 8,
 -1,
 2,
 1,
 -1,
 1,
 2,
 1,
 1,
 1,
 3,
 5,
 -1,
 1,
 5,
 -1,
 1,
 2,
 2,
 2,
 1,
 -1,
 -1,
 1,
 2,
 1,
 1,
 1,
 -1,
 -1,
 1,
 2,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 2,
 -1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 2,
 -1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 2,
 1,
 1,
 2,
 -1,
 1,
 1,
 1,
 1,
 3,
 1,
 -1,
 1,
 1,
 1,
 2,
 1,
 3,
 9,
 -1,
 -1,
 1,
 1,
 2,
 3,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,


In [36]:
notfound_indexes = list()
for i in range(len(ranks)):
    if ranks[i] != -1:
        continue
    notfound_indexes.append(i)

In [37]:
for index in notfound_indexes:
    print(index, len(algo_results[index]["graph"]), algo_results[index]["sparql_query"])

5 3 SELECT DISTINCT ?uri WHERE {?uri <http://dbpedia.org/ontology/order> <http://dbpedia.org/resource/Bacillales> . ?uri <http://dbpedia.org/property/domain> <http://dbpedia.org/resource/Bacteria>  . ?uri <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Bacteria>}
6 2 SELECT DISTINCT ?uri WHERE { ?uri <http://dbpedia.org/ontology/formerTeam> <http://dbpedia.org/resource/Indianapolis_Colts> . ?uri <http://dbpedia.org/ontology/formerTeam> <http://dbpedia.org/resource/Carolina_Panthers> . }
7 2 SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/headquarters> <http://dbpedia.org/resource/Gold_Base> . ?uri <http://dbpedia.org/ontology/religion> ?x  . }
9 2 SELECT DISTINCT ?uri WHERE {?uri <http://dbpedia.org/property/distributor> <http://dbpedia.org/resource/HBO>  . ?uri <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/TelevisionShow>}
13 2 SELECT DISTINCT ?uri WHERE {?uri <http://dbpedia.org/property/music> <http://dbped

In [38]:
for index in notfound_indexes:
    if len(algo_results[index]["graph"]) == 3:
        continue
    print(index, len(algo_results[index]["graph"]), algo_results[index]["sparql_query"])

6 2 SELECT DISTINCT ?uri WHERE { ?uri <http://dbpedia.org/ontology/formerTeam> <http://dbpedia.org/resource/Indianapolis_Colts> . ?uri <http://dbpedia.org/ontology/formerTeam> <http://dbpedia.org/resource/Carolina_Panthers> . }
7 2 SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/headquarters> <http://dbpedia.org/resource/Gold_Base> . ?uri <http://dbpedia.org/ontology/religion> ?x  . }
9 2 SELECT DISTINCT ?uri WHERE {?uri <http://dbpedia.org/property/distributor> <http://dbpedia.org/resource/HBO>  . ?uri <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/TelevisionShow>}
13 2 SELECT DISTINCT ?uri WHERE {?uri <http://dbpedia.org/property/music> <http://dbpedia.org/resource/Laurie_Johnson>  . ?uri <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Film>}
54 2 SELECT DISTINCT ?uri WHERE {?uri <http://dbpedia.org/ontology/instrument> <http://dbpedia.org/resource/Rickenbacker>  . ?uri <http://www.w3.org/1999/02/22-rdf-synta

In [39]:
for index in notfound_indexes:
    if "?x" not in algo_results[index]["sparql_query"]:
        continue
    print(index, len(algo_results[index]["graph"]), algo_results[index]["sparql_query"])

7 2 SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/headquarters> <http://dbpedia.org/resource/Gold_Base> . ?uri <http://dbpedia.org/ontology/religion> ?x  . }
14 3 SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/ontology/developer> <http://dbpedia.org/resource/Blizzard_Entertainment> . ?x <http://dbpedia.org/property/genre> ?uri  . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/VideoGame>}
22 3 SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/firstTeam> <http://dbpedia.org/resource/Williams_Grand_Prix_Engineering> . ?x <http://dbpedia.org/ontology/poleDriver> ?uri  . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/GrandPrix>}
23 3 SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/ontology/layout> <http://dbpedia.org/resource/Front-engine,_four-wheel-drive_layout> . ?x <http://dbpedia.org/ontology/layout> ?uri  . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/on

In [16]:
notfound_indexes[14]

39

In [26]:
for item in algo_results[236]["explanation"].keys():
    print(item, len(algo_results[236]["explanation"][item]))

forward 7
invalids 320
backward 2
backward_forward 309
forward_backward 0


In [27]:
print(len(algo_results[236]["result"]), len(algo_results[236]["result_urlonly"]))

13 13


In [29]:
algo_results[236]["sparql_query"]

'SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Aegean_Sea> <http://dbpedia.org/ontology/outflow> ?x . ?x <http://dbpedia.org/property/cities> ?uri  . }'

In [28]:
algo_results[236]["explanation"]["backward_forward"]

[[[['?ak', 'http://dbpedia.org/property/location', '?uri'],
   ['?ak',
    'http://dbpedia.org/property/stadium',
    'http://dbpedia.org/resource/Stade_Mohamed_V']],
  539],
 [[['?ar', 'http://dbpedia.org/ontology/headquarter', '?uri'],
   ['?ar',
    'http://dbpedia.org/property/language',
    'http://dbpedia.org/resource/French_language']],
  693],
 [[['?am', 'http://dbpedia.org/ontology/headquarter', '?uri'],
   ['?am',
    'http://dbpedia.org/ontology/language',
    'http://dbpedia.org/resource/French_language']],
  769],
 [[['?ay', 'http://dbpedia.org/property/location', '?uri'],
   ['?ay',
    'http://dbpedia.org/property/location',
    'http://dbpedia.org/resource/Casablanca']],
  945],
 [[['?ap', 'http://dbpedia.org/ontology/birthPlace', '?uri'],
   ['?ap', 'http://purl.org/dc/elements/1.1/description', 'Artist']],
  968],
 [[['?ah', 'http://dbpedia.org/property/city', '?uri'],
   ['?ah',
    'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
    'http://dbpedia.org/ontology/S

In [10]:
x["result_urlonly"]

['http://dbpedia.org/resource/California',
 'http://dbpedia.org/resource/San_Francisco',
 'http://dbpedia.org/resource/Chennai',
 'http://dbpedia.org/resource/Tamil_Nadu',
 'http://dbpedia.org/resource/Seattle',
 'http://dbpedia.org/resource/Atlanta',
 'http://dbpedia.org/resource/Westminster,_Colorado',
 'http://dbpedia.org/resource/San_Jose,_California']

In [8]:
x["explanation"]

defaultdict(list,
            {'forward': [['?uri',
               'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
               'http://dbpedia.org/ontology/Settlement'],
              ['?uri',
               'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
               'http://dbpedia.org/ontology/Location'],
              ['?uri',
               'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
               'http://dbpedia.org/dbtax/Establishment'],
              ['?uri',
               'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
               'http://dbpedia.org/ontology/PopulatedPlace'],
              ['?uri',
               'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
               'http://dbpedia.org/ontology/Place']],
             'invalids': [[['?aq',
                'http://dbpedia.org/ontology/deathPlace',
                '?uri'],
               ['?aq', 'http://dbpedia.org/ontology/deathYear', '1986']],
              [['?ar', 'http://dbpedia.org/ont

In [22]:
import numpy as np
from tqdm import tqdm


def graph_eval_ranking_first(gold_querry, pred_querries):
    # Figyelj arra amikor több azonos result size-ú query van, az MR és MRR nem kéne csökkenjen? - DONE

    querries_with_count_sorted = sorted(pred_querries, key=lambda x: x[1])
    querries_with_count_sorted_selection = [item[0] for item in querries_with_count_sorted]
    rank_counter = 0
    for i, element in enumerate(querries_with_count_sorted):
        if i == 0:
            rank_counter += 1
        elif querries_with_count_sorted[i - 1][1] != element[1]:
            rank_counter += 1

        overlap = query_overlap(gold_querry, querries_with_count_sorted_selection[i])
        if overlap:
            return rank_counter

    return -1


def get_mr_metrics(ranks, hits=[1, 5, 10]):
    mrr = np.mean([1 / item for item in ranks])
    mr = np.mean(ranks)

    hits_metrics = list()
    for hit in hits:
        hits_metrics.append(len(list(filter(lambda x: x < hit, ranks))) / len(ranks))

    return mr, mrr, hits_metrics


def graph_eval_ranking_ds(ds, pred_column="explanation", recalculate=True):
    ranks = list()

    for record in tqdm(ds):
        # Filter out Nones
        if isinstance(record[pred_column], list):
            continue
        if recalculate:
            querries_with_count = get_query_counts(record[pred_column])
        else:
            querries_with_count = record[pred_column]

        record_rank = graph_eval_ranking_first(record["graph"], querries_with_count)
        ranks.append(record_rank)

    metrics = get_mr_metrics(ranks, )
    return metrics


In [64]:
ranks = list()

ds = algo_results
pred_column = "explanation"
recalculate = True

for record in tqdm(ds):
    # Filter out Nones
    if isinstance(record[pred_column], list):
        continue
    if recalculate:
        querries_with_count = get_query_counts(record[pred_column])
    else:
        querries_with_count = record[pred_column]

    record_rank = graph_eval_ranking_first(record["graph"], querries_with_count)
    ranks.append(record_rank)

metrics = get_mr_metrics(ranks, )

100%|█████████████████████████████████████████| 250/250 [00:10<00:00, 23.16it/s]


In [66]:
type(record[pred_column])

collections.defaultdict

In [24]:
metrics

(8.457831325301205,
 0.35793222705399697,
 [0.17670682730923695, 0.8674698795180723, 0.891566265060241])

In [56]:
graph_eval_ranking_ds(algo_results, pred_column="explanation", recalculate=True)

100%|█████████████████████████████████████████| 250/250 [00:11<00:00, 21.56it/s]


(10.487804878048781,
 0.6493908513972939,
 [0.40963855421686746, 0.7068273092369478, 0.7188755020080321],
 44)

In [7]:
from evaluation.evaluate_script import graph_eval_ranking_ds, graph_eval_ranking_first, get_query_counts, query_overlap, get_mr_metrics

ImportError: cannot import name 'graph_eval_ranking_ds' from 'evaluation.evaluate_script' (/home/kardosp/continuethelist/notebooks/evaluation/evaluate_script.py)

In [8]:
ranks = list()

ds = algo_results
pred_column = "explanation"
recalculate = True

for record in tqdm(ds):
    # Filter out Nones
    if isinstance(record[pred_column], list):
        continue
    if recalculate:
        querries_with_count = get_query_counts(record[pred_column])
    else:
        querries_with_count = record[pred_column]

    record_rank = graph_eval_ranking_first(record["graph"], querries_with_count)
    ranks.append(record_rank)

metrics = get_mr_metrics(ranks, )

NameError: name 'algo_results' is not defined

In [48]:
metrics

(10.949704142011834,
 0.6470454169285325,
 [0.0, 0.6953316953316954, 0.7199017199017199],
 138)

In [13]:
record = algo_results[3]

querries_with_count = get_query_counts(record["explanation"])
querries_with_count_sorted = sorted(querries_with_count, key=lambda x: x[1])
querries_with_count_sorted_selection = [item[0] for item in querries_with_count_sorted]

In [75]:
metrics = graph_eval_ranking(record["graph"], querries_with_count_sorted_selection)

In [14]:
querries_with_count_sorted

[[[['http://dbpedia.org/resource/Skull_Gang',
    'http://dbpedia.org/ontology/bandMember',
    '?uri']],
  12],
 [[['http://dbpedia.org/resource/Skull_Gang',
    'http://dbpedia.org/property/currentMembers',
    '?uri']],
  12],
 [[['?ac', 'http://dbpedia.org/ontology/bandMember', '?uri'],
   ['?ac',
    'http://dbpedia.org/ontology/associatedMusicalArtist',
    'http://dbpedia.org/resource/Jim_Jones_(rapper)']],
  12],
 [[['?ah', 'http://dbpedia.org/ontology/bandMember', '?uri'],
   ['?ah',
    'http://dbpedia.org/ontology/bandMember',
    'http://dbpedia.org/resource/Yung_Ja']],
  12],
 [[['?am', 'http://dbpedia.org/ontology/bandMember', '?uri'],
   ['?am',
    'http://dbpedia.org/ontology/bandMember',
    'http://dbpedia.org/resource/Robert_De_Niro']],
  12],
 [[['?an', 'http://dbpedia.org/ontology/bandMember', '?uri'],
   ['?an',
    'http://dbpedia.org/ontology/associatedBand',
    'http://dbpedia.org/resource/Jim_Jones_(rapper)']],
  12],
 [[['?ap', 'http://dbpedia.org/ontology/

In [10]:
for i in range(len(querries_with_count_sorted_selection)):
    print(i, query_overlap(record["graph"], querries_with_count_sorted_selection[i]))

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 True
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
21 False
22 False
23 False
24 False
25 False
26 False
27 False
28 False
29 False
30 False
31 False
32 False
33 False
34 False
35 False
36 False
37 False
38 False
39 False
40 False
41 False
42 False
43 False
44 False
45 False
46 False
47 False
48 False
49 False
50 False
51 False
52 False
53 False
54 False
55 False
56 False
57 False
58 False
59 False
60 False
61 False
62 False
63 False
64 False
65 False
66 False
67 False
68 False
69 False
70 False
71 False
72 False
73 False
74 False
75 False
76 False
77 False
78 False
79 False
80 False
81 False
82 False
83 False
84 False
85 False
86 False
87 False
88 False
89 False
90 False
91 False
92 False
93 False
94 False
95 False
96 False
97 False
98 False
99 False
100 False
101 False
102 False
103 False
104 False
105 False
106 False
107 False
108 False
109 False
110 False
1

In [8]:
i = 1
print(i, query_overlap(record["graph"], querries_with_count_sorted_selection[i]))

Rósz
['http://dbpedia.org/resource/Skull_Gang', 'http://dbpedia.org/property/currentMembers', '?uri'] ['?x', 'http://dbpedia.org/property/currentMembers', 'http://dbpedia.org/resource/Hynief'] 0
Rósz
['http://dbpedia.org/resource/Skull_Gang', 'http://dbpedia.org/property/currentMembers', '?uri'] ['?x', 'http://dbpedia.org/ontology/bandMember', '?uri'] 0
Rósz
['http://dbpedia.org/resource/Skull_Gang', 'http://dbpedia.org/property/currentMembers', '?uri'] ['?x', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://dbpedia.org/ontology/Band'] 0
1 False


In [81]:
querries_with_count_sorted_selection[9]

[['?ax', 'http://dbpedia.org/ontology/bandMember', '?uri'],
 ['?ax',
  'http://dbpedia.org/property/currentMembers',
  'http://dbpedia.org/resource/Hynief']]

In [102]:
query_overlap(record["graph"], querries_with_count_sorted_selection[9])

Rósz
['?ax', 'http://dbpedia.org/ontology/bandMember', '?uri'] ['?x', 'http://dbpedia.org/property/currentMembers', 'http://dbpedia.org/resource/Hynief'] 1
excl ind:  [True, True]
all_triples:  [['?x', '<http://dbpedia.org/property/currentMembers>', '<http://dbpedia.org/resource/Hynief>'], ['?x', '<http://dbpedia.org/ontology/bandMember>', '?uri'], ['?x', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://dbpedia.org/ontology/Band>']]
smaller  [['?ax', 'http://dbpedia.org/ontology/bandMember', '?uri'], ['?ax', 'http://dbpedia.org/property/currentMembers', 'http://dbpedia.org/resource/Hynief']]


True

In [5]:
from evaluation.evaluate_script import graph_eval_ranking_ds_nodes, graph_eval_ranking_first_nodes, get_query_counts, get_mr_metrics_triples

In [6]:
graph_eval_ranking_ds_nodes(ds, "explanation")

NameError: name 'ds' is not defined

In [13]:
from tqdm import tqdm

ds = algo_results
pred_column = "explanation"
ranks = list()

for record in tqdm(ds):
    # Filter out Nones
    if isinstance(record[pred_column], list):
        continue

    record_rank = graph_eval_ranking_first_nodes(record["graph"], record[pred_column])
    ranks.append(record_rank)


100%|█████████████████████████████████████████| 250/250 [04:32<00:00,  1.09s/it]


In [14]:
metrics = get_mr_metrics_triples(ranks)

In [15]:
ranks

[2,
 2,
 1,
 1,
 3,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 2,
 -1,
 -1,
 -1,
 1,
 -1,
 4,
 2,
 4,
 -1,
 1,
 28,
 118,
 1,
 1,
 -1,
 4,
 -1,
 2,
 1,
 2,
 1,
 3,
 2,
 -1,
 335,
 6,
 2,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 2,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 2,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 4,
 -1,
 1,
 1,
 1,
 2,
 -1,
 1,
 2,
 -1,
 2,
 1,
 5,
 -1,
 -1,
 1,
 -1,
 -1,
 35,
 2,
 3,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 2,
 -1,
 13,
 1,
 2,
 -1,
 1,
 1,
 3,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 2,
 -1,
 2,
 1,
 -1,
 -1,
 1,
 2,
 1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 19,
 1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 2,
 -1,
 25,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 2,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 2,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 2,
 5,
 -1,
 -1,
 1,
 -1,
 3,
 1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 3,
 10,
 -1,
 -1,
 -1,
 -1,
 3,
 -1,
 1,
 2,
 -1,
 1,
 1,
 2,
 1,
 -1

In [16]:
metrics

(5.825174825174825,
 0.7561983002211421,
 [0.357429718875502, 0.5341365461847389, 0.5421686746987951],
 106)

In [201]:
record = algo_results[6]

rank1  = graph_eval_ranking_first_nodes(record["graph"], record["explanation"], True)

In [202]:
record["graph"]

[['?uri',
  '<http://dbpedia.org/ontology/formerTeam>',
  '<http://dbpedia.org/resource/Indianapolis_Colts>'],
 ['?uri',
  '<http://dbpedia.org/ontology/formerTeam>',
  '<http://dbpedia.org/resource/Carolina_Panthers>']]

In [203]:
rank1

(-1,
 [[[['?uri',
     'http://dbpedia.org/ontology/formerTeam',
     'http://dbpedia.org/resource/Carolina_Panthers']],
   652],
  [[['?uri',
     'http://dbpedia.org/ontology/formerTeam',
     'http://dbpedia.org/resource/Indianapolis_Colts']],
   826],
  [[['?uri',
     'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
     'http://dbpedia.org/ontology/AmericanFootballPlayer']],
   21684],
  [[['?uri',
     'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
     'http://dbpedia.org/ontology/GridironFootballPlayer']],
   25751],
  [[['?uri',
     'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
     'http://dbpedia.org/ontology/Athlete']],
   371804],
  [[['?uri',
     'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
     'http://xmlns.com/foaf/0.1/Person']],
   1518282],
  [[['?uri',
     'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
     'http://dbpedia.org/ontology/Person']],
   1767272],
  [[['?uri',
     'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
     'http://d

In [189]:
from query.query_functions2 import get_result_from_triples
len(get_result_from_triples(record["graph"]))

39

In [169]:
gold_graph_with_nodes = [record["graph"], get_result_from_triples(record["graph"])]
pred_graphs_with_counts = get_query_counts_recalculate(record["explanation"])
pred_graphs_with_counts_sorted = sorted(pred_graphs_with_counts, key=lambda x: x[1])

rank = 0
prev_size = -1
for explanation in pred_graphs_with_counts_sorted:
    if explanation[1] != prev_size:
        rank += 1
        prev_size = explanation[1]
    print("SIZE: ", explanation[1], len(gold_graph_with_nodes[1]))
    if len(gold_graph_with_nodes[1]) == explanation[1]:

        explanation_nodes = get_result_from_triples(explanation[0])
        # if filter_name == "url":
        #     explanation_nodes = list(filter(lambda x: "http" in x, explanation_nodes))

        print("Explanation:", explanation_nodes)
        print("GOLD:", gold_graph_with_nodes[1])
        if len(set(gold_graph_with_nodes[1]).intersection(set(explanation_nodes))) == explanation[1]:
            break

        if len(gold_graph_with_nodes[1]) > explanation[1]:
            rank = -1
            break


SIZE:  25 39
SIZE:  30 39
SIZE:  41 39
SIZE:  43 39
SIZE:  403 39
SIZE:  418 39
SIZE:  554 39
SIZE:  567 39
SIZE:  2252 39
SIZE:  2904 39
SIZE:  2906 39
SIZE:  2999 39
SIZE:  3769 39
SIZE:  3771 39
SIZE:  41758 39
SIZE:  42644 39
SIZE:  48900 39
SIZE:  58787 39
SIZE:  1767272 39


In [24]:
from evaluation.evaluate_script import get_query_counts
from evaluation.evaluation_script_v2 import MAIN_node_eval_ds, MAIN_node_eval_ds_aggregated
x = algo_results[0]

counts = get_query_counts(x["explanation"])

In [10]:
from excludes.filters import filter_url, filter_none
filter_function = filter_url

In [17]:
res = MAIN_node_eval_ds(algo_results, filter_function, top_funcs=top_funcs)

Node eval TOP3


100%|█████████████████████████████████████████| 250/250 [00:20<00:00, 11.95it/s]

Node eval TOP3 Done!


In [19]:
res

[[[0.3888888888888889, 0.875, 0.5384615384615385],
  [0.4, 1.0, 0.5714285714285715]],
 [[0.7241379310344828, 0.984375, 0.8344370860927153],
  [0.6956521739130435, 1.0, 0.8205128205128205]],
 [[1.0, 1.0, 1.0],
  [0.6666666666666666, 0.7272727272727273, 0.6956521739130435],
  [0.2702702702702703, 0.9090909090909091, 0.4166666666666667]],
 [[1.0, 1.0, 1.0]],
 [[0.875, 1.0, 0.9333333333333333],
  [0.7241379310344828, 1.0, 0.8400000000000001],
  [0.5121951219512195, 1.0, 0.6774193548387097]],
 [[0.20673076923076922, 1.0, 0.3426294820717131],
  [0.2037914691943128, 1.0, 0.33858267716535434]],
 [[0.09662576687116564, 1.0, 0.17622377622377622],
  [0.07627118644067797, 1.0, 0.14173228346456693]],
 [[1.0, 0.8709677419354839, 0.9310344827586207],
  [1.0, 0.967741935483871, 0.9836065573770492]],
 [[1.0, 1.0, 1.0],
  [0.012844909609895337, 0.5094339622641509, 0.025058004640371227]],
 [[0.2926829268292683, 1.0, 0.45283018867924524],
  [0.27325581395348836, 0.9791666666666666, 0.4272727272727273]],
 

In [25]:
from evaluation.query_selectors import select_min, select_xth, select_thresholded

a = select_min
b = lambda x: select_xth(x, 2)
c = lambda x: select_thresholded(x, 100)
top_funcs = [a, b, c]


res = MAIN_node_eval_ds_aggregated(algo_results, filter_function, top_funcs=top_funcs)

Node eval TOP3


100%|█████████████████████████████████████████| 250/250 [00:21<00:00, 11.43it/s]

Node eval TOP3 Done!


In [31]:
res

{'precision': 0.8254466576078908,
 'recall': 0.9105231510686616,
 'f1': 0.8369611167517508,
 'best_counter': defaultdict(int, {1: 40, 0: 198, 2: 11})}

In [32]:
from excludes.filters import filter_url, filter_none
from evaluation.evaluation_script_v2 import MAIN_node_eval_ds_aggregated, MAIN_graph_ranking_on_nodes
from evaluation.query_selectors import select_min, select_xth, select_thresholded

a = select_min
b = lambda x: select_xth(x, 2)
c = lambda x: select_thresholded(x, 100)
top_funcs = [a, b, c]

filter_function = filter_url

output_dict = MAIN_node_eval_ds_aggregated(algo_results, filter_function, pred_column="output", gold_column="result_urlonly", top_funcs=top_funcs)

if "explanation" in algo_results[0]:
    # graph_eval_relaxed_score = graph_eval_relaxed_ds(algo_results)
    rank_metrics = MAIN_graph_ranking_on_nodes(algo_results, "explanation", False)

    for k, v in rank_metrics.items():
        output_dict[k] = v

output_dict

Node eval TOP3


100%|█████████████████████████████████████████| 250/250 [00:21<00:00, 11.71it/s]


Node eval TOP3 Done!


100%|█████████████████████████████████████████| 250/250 [00:17<00:00, 13.96it/s]


{'precision': 0.8254466576078908,
 'recall': 0.9105231510686616,
 'f1': 0.8369611167517508,
 'best_counter': dict_items([(1, 40), (0, 198), (2, 11)]),
 'MR': 1.7515527950310559,
 'MRR': 0.8409286969069578,
 'rank_missing': 88,
 'hits@1': 0.46586345381526106,
 'hits@5': 0.6305220883534136,
 'hits@10': 0.6385542168674698}

In [46]:
for index in notfound_indexes:
    if len(algo_results[index]["graph"]) == 3:
        continue
    if "?x" not in algo_results[index]["sparql_query"] :
        continue
    print(index, len(algo_results[index]["graph"]), algo_results[index]["sparql_query"])

7 2 SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/headquarters> <http://dbpedia.org/resource/Gold_Base> . ?uri <http://dbpedia.org/ontology/religion> ?x  . }
128 2 SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/leaderName> <http://dbpedia.org/resource/David_Miscavige> . ?uri <http://dbpedia.org/ontology/religion> ?x  . }
196 2 SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/state> <http://dbpedia.org/resource/Maharashtra> . ?x <http://dbpedia.org/ontology/sport> ?uri  . }
247 2 SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/schooltype> <http://dbpedia.org/resource/Private_school> . ?x <http://dbpedia.org/ontology/sport> ?uri  . }


In [44]:
for k, v in algo_results[14]["explanation"].items():
    print(k, len(v))

backward 0
invalids 10
backward_forward 74
forward 0
forward_backward 0
backward_backward 0
forward_forward 0


In [45]:
algo_results[14]["explanation"]["backward_forward"]

[[[['?an', 'http://dbpedia.org/ontology/genre', '?uri'],
   ['?an',
    'http://dbpedia.org/ontology/developer',
    'http://dbpedia.org/resource/Blizzard_Entertainment']],
  43],
 [[['?am', 'http://dbpedia.org/property/genre', '?uri'],
   ['?am',
    'http://dbpedia.org/ontology/developer',
    'http://dbpedia.org/resource/Blizzard_Entertainment']],
  50],
 [[['?ak', 'http://dbpedia.org/ontology/genre', '?uri'],
   ['?ak',
    'http://dbpedia.org/ontology/computingPlatform',
    'http://dbpedia.org/resource/IPod_Touch']],
  54],
 [[['?ay', 'http://dbpedia.org/ontology/genre', '?uri'],
   ['?ay',
    'http://dbpedia.org/property/platforms',
    'http://dbpedia.org/resource/Windows_95']],
  77],
 [[['?ap', 'http://dbpedia.org/ontology/genre', '?uri'],
   ['?ap',
    'http://dbpedia.org/ontology/computingPlatform',
    'http://dbpedia.org/resource/Windows_95']],
  83],
 [[['?ab', 'http://dbpedia.org/property/genre', '?uri'],
   ['?ab',
    'http://dbpedia.org/property/platforms',
    'ht

In [49]:
from query.query_functions2 import count_all_triplets_from_ro
record = algo_results[247]
for seed in record["seed"]:
    print(seed, count_all_triplets_from_ro(seed, "http://dbpedia.org/ontology/sport"))

http://dbpedia.org/resource/Badminton [{'callret-0': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '110'}}]
http://dbpedia.org/resource/Basketball [{'callret-0': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '427'}}]
http://dbpedia.org/resource/Table_tennis [{'callret-0': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '93'}}]
http://dbpedia.org/resource/Field_hockey [{'callret-0': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '24'}}]


In [ ]:
from tqdm import tqdm
from evaluation.evaluation_script_v2 import MAIN_node_eval_ds_aggregated

pred_column = "explanation"
ranks = list()
extra_info = True

query_sizes = list()
for record in tqdm(algo_results):
    # Filter out Nones
    if isinstance(record[pred_column], list):
        continue
    if extra_info:
        record_rank, _, query_size_single = graph_ranking_first_nodes(record["graph"],
                                                                      record[pred_column],
                                                                      extra_info)
        query_sizes.append(query_size_single)
    else:
        record_rank = graph_ranking_first_nodes(record["graph"], record[pred_column])
    ranks.append(record_rank)


In [8]:
from evaluation.evaluation_script_v2 import MAIN_node_eval_ds_aggregated
from excludes.filters import filter_url
from evaluation.query_selectors import select_min, select_xth, select_thresholded, select_average

a = select_min
# b = lambda x: select_xth(x, 2)
b = lambda x: select_average(x, 100)
c = lambda x: select_thresholded(x, 100)
top_funcs = [a, b, c]

filter_function = filter_url
output_dict = MAIN_node_eval_ds_aggregated(algo_results, filter_function, pred_column="output",
                                           gold_column="result_urlonly", top_funcs=top_funcs)

Node eval TOP3


  1%|▌                                          | 3/250 [00:00<00:18, 13.38it/s]/home/kardosp/miniconda3/envs/continue/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/kardosp/miniconda3/envs/continue/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|█████████████████████████████████████████| 250/250 [00:12<00:00, 20.42it/s]

Node eval TOP3 Done!


In [9]:
output_dict

{'precision': 0.8920805218714133,
 'recall': 0.9425905831668021,
 'f1': 0.8983635619972474,
 'best_counter': [(1, 42), (0, 203), (2, 5)],
 'triple_match': 0.42066666666666663}